In [65]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jieba
from jieba import lcut, analyse, Tokenizer
from wordcloud import WordCloud, STOPWORDS
from gensim.models import word2vec
from gensim.models.keyedvectors import KeyedVectors
from collections import Counter
from datetime import datetime

In [2]:
def Openfile(path):
    f = open(path, "r", encoding="utf-8")
    result = f.read()
    f.close()
    return result
def Openfile_sp(path):
    f = open(path, "r", encoding="utf-8")
    result = f.readlines()
    f.close()
    return result

In [47]:
# open file
stopwords_file = 'datafile/stop_words.txt'
jieba.set_dictionary('./dict.txt.big.txt')
jieba.load_userdict('datafile/user_dict.txt')
jieba.analyse.set_stop_words('datafile/stop_words.txt')

evaair_file = 'datafile/EvaAir_text.csv'
eva_sta = 'datafile/EvaAir_sta.csv'
df = pd.read_csv(evaair_file)

Building prefix dict from D:\WORK5\TechProject\TechProject2019\dict.txt.big.txt ...
Loading model from cache C:\Users\ASUS\AppData\Local\Temp\jieba.u15c153cb84f92c395787873e58f7016c.cache
Loading model cost 1.057 seconds.
Prefix dict has been built succesfully.


In [4]:
dec = Openfile_sp(stopwords_file)
stopwords = set()
for i in range(len(dec)):
    stopwords.add(dec[i].strip())

In [5]:
df1 = df['text'].tolist()
for i in range(len(df1)):
    if isinstance(df1[i], float):
        df1[i] = str(df1[i])
evaair_text = []
for i in range(len(df1)):
    evaair_text.append(df1[i].strip())

In [43]:
result = []
for i in range(len(evaair_text)):
    r = lcut(evaair_text[i], cut_all=False, HMM=False)
    result.append(r)
    
adj_result = []
for i in range(len(result)):
    par = []
    for j in range(len(result[i])):
        if len(result[i][j]) > 1:
            if result[i][j] not in stopwords:
                par.append(result[i][j])
    adj_result.append(par)

In [44]:
# frequency counter
c = Counter()
for i in range(len(adj_result)):
    for j in adj_result[i]:
        if len(j) > 1:
            c[j] += 1
freq_result = c.most_common(100)

df_freq = pd.DataFrame(freq_result)
df_freq.to_csv('datafile/evaair_freqkey.csv')

In [77]:
E_model = word2vec.Word2Vec(adj_result, size=200, min_count=10)
E_model.save("model/EvaAir_model.bin")

In [101]:
model = word2vec.Word2Vec.load("model/EvaAir_model.bin")
word_vectors = model.wv

In [102]:
res = word_vectors.wv.most_similar('長榮航空', topn = 20)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [103]:
res

[('一同', 0.9990308284759521),
 ('美好', 0.9988092184066772),
 ('期許', 0.9987189173698425),
 ('安全', 0.9986463785171509),
 ('飛行', 0.9986349940299988),
 ('體驗', 0.9986276626586914),
 ('未來', 0.9985326528549194),
 ('邀請', 0.9984458684921265),
 ('旅程', 0.998437762260437),
 ('今天', 0.9982931613922119),
 ('旅行', 0.9982914924621582),
 ('影片', 0.9982848763465881),
 ('設計', 0.9982648491859436),
 ('姐妹', 0.9981586933135986),
 ('舒適', 0.9981505870819092),
 ('加入', 0.9981184005737305),
 ('各位', 0.9981131553649902),
 ('世界', 0.9979893565177917),
 ('機上', 0.9979846477508545),
 ('希望', 0.997948169708252)]

In [8]:
# test part
key_word = 'datafile/evaair_keywords.csv'
df = pd.read_csv(key_word)
df1 = df['key'].tolist()
key = set()
for i in range(len(df1)):
    key.add(df1[i])
    
key_result = []
for i in range(len(adj_result)):
    par = []
    for j in range(len(adj_result[i])):
        if adj_result[i][j] in key:
            par.append(adj_result[i][j])
    key_result.append(par)

In [34]:
# key association 

key_a = '旅遊'
key_b = df1
instances = 1665
assoc_result = []
for j in range(len(key_b)):
    counter = 0
    r_counter = 0
    out_put = []
    for i1 in range(len(key_result)):
        if key_a in key_result[i1]:
            r_counter += 1
            continue
    for i2 in range(len(key_result)):
        if ((key_result[i2].count(key_a) > 0) and (key_result[i2].count(key_b[j]) > 0)):
            counter += 1
    result = counter / r_counter
    out_put.append(key_b[j])
    out_put.append(result)
    assoc_result.append(out_put)

assoc_result_adj = []
for i in range(len(assoc_result)):
    for j in range(len(assoc_result[i])):
        if assoc_result[i][1] >= 0.1:
            assoc_result_adj.append(assoc_result[i])
            break

new_assoc_result = sorted(assoc_result_adj, key=lambda I:I[1], reverse=True)
new_assoc_result

[['旅遊', 1.0],
 ['長榮航空', 0.5849056603773585],
 ['小潔', 0.48427672955974843],
 ['http', 0.48427672955974843],
 ['長榮', 0.3836477987421384],
 ['機票', 0.29559748427672955],
 ['活動', 0.2830188679245283],
 ['EVAAir', 0.24528301886792453],
 ['旅客', 0.24528301886792453],
 ['獲得', 0.2389937106918239],
 ['機會', 0.23270440251572327],
 ['行程', 0.23270440251572327],
 ['服務', 0.22641509433962265],
 ['可以', 0.22012578616352202],
 ['體驗', 0.2138364779874214],
 ['分享', 0.20754716981132076],
 ['即日起', 0.20125786163522014],
 ['機場', 0.18238993710691823],
 ['旅行', 0.18238993710691823],
 ['航班', 0.1761006289308176],
 ['粉絲', 0.16981132075471697],
 ['全球', 0.16981132075471697],
 ['出國', 0.16352201257861634],
 ['搭乘', 0.15723270440251572],
 ['時間', 0.15723270440251572],
 ['世界', 0.15723270440251572],
 ['優惠', 0.1509433962264151],
 ['出發', 0.1509433962264151],
 ['台北', 0.1509433962264151],
 ['2016', 0.1509433962264151],
 ['期間', 0.1509433962264151],
 ['資訊', 0.1509433962264151],
 ['最佳', 0.1509433962264151],
 ['EVA', 0.14465408805031446

In [9]:
E_test_model = word2vec.Word2Vec(key_result, size=300, min_count=10)
E_test_model.save("model/Evaair_key_model.bin")

In [10]:
key_model = word2vec.Word2Vec.load("model/Evaair_key_model.bin")
word_vectors = key_model.wv

In [11]:
res = word_vectors.wv.most_similar('長榮航空', topn = 30)

D:\Program Files\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [12]:
res

[('透過', 0.9992537498474121),
 ('感謝', 0.9991846084594727),
 ('邀請', 0.9989438056945801),
 ('旅程', 0.9989300966262817),
 ('舒適', 0.9988234043121338),
 ('飛行', 0.9987283945083618),
 ('夢想', 0.9985811710357666),
 ('體驗', 0.998435378074646),
 ('歡迎', 0.9984322190284729),
 ('影片', 0.9982989430427551),
 ('EVAAir', 0.9982054233551025),
 ('美麗', 0.9982041716575623),
 ('未來', 0.998146653175354),
 ('開始', 0.9981433749198914),
 ('提供', 0.9981133341789246),
 ('世界', 0.9980989694595337),
 ('特別', 0.9977622628211975),
 ('歡樂', 0.9976341724395752),
 ('準備', 0.9976314902305603),
 ('即將', 0.9975811243057251),
 ('這個', 0.9973492622375488),
 ('777', 0.9973303079605103),
 ('可以', 0.9973263740539551),
 ('設計', 0.9972262978553772),
 ('旅行', 0.9972051382064819),
 ('詳情', 0.9971486330032349),
 ('旅遊', 0.9970923662185669),
 ('小潔', 0.9968395233154297),
 ('驚喜', 0.99676513671875),
 ('美國', 0.9967626929283142)]

In [37]:
# text analysis
product = set()
price = set()
place = set()
depart = set()
company = set()
question = set()
prize = set()


product_key = ['航班', '航線', '班機', '飛航', '服務', '機上', 'Kitty', '搭乘', '彩繪機', 
               '體驗', '787', '經濟艙']
price_key = ['TWD', '優惠']
place_key = ['美國', '東京', '日本', '沖繩', '香港']
depart_key = ['台北', '高雄']
prize_key = ['免費', '機會', '獲得', '得獎', '得獎者', '抽獎', '好康', '參加']
question_key = ['?', '？']


for i in range(len(product_key)):
    product.add(product_key[i])
for i in range(len(price_key)):
    price.add(price_key[i])
for i in range(len(place_key)):
    place.add(place_key[i])
for i in range(len(depart_key)):
    depart.add(depart_key[i])
for i in range(len(question_key)):
    question.add(question_key[i])
for i in range(len(prize_key)):
    prize.add(prize_key[i])

In [46]:
counter = 0
for i in range(len(key_result)):
    for j in range(len(key_result[i])):
        if key_result[i][j] in prize:
            counter += 1
            break
text_ana_res = counter / instances
text_ana_res


0.2816816816816817

In [45]:
counter1 = 0
for i in range(len(result)):
    for j in range(len(result[i])):
        if result[i][j] in question:
            counter1 += 1
            break
text_ana_res1 = counter1 / instances
text_ana_res1

0.33153153153153153

In [88]:
output_res = []
text_df = pd.read_csv('datafile/EvaAir_text.csv')
ID = text_df['no'].tolist()
for i in range(len(result)):
    n_ID = ID[i]
    product_count = 0
    price_count = 0
    place_count = 0
    depart_count = 0
    prize_count = 0
    question_count = 0
    for j in range(len(result[i])):
        if result[i][j] in product:
            product_count += 1
            continue
        if result[i][j] in price:
            price_count += 1
            continue
        if result[i][j] in place:
            place_count += 1
            continue
        if result[i][j] in depart:
            depart_count += 1
            continue
        if result[i][j] in prize:
            prize_count += 1
            continue
        if result[i][j] in question:
            question_count += 1
            continue
        process = [n_ID, product_count, price_count, place_count, 
                   depart_count, prize_count, question_count]
    output_res.append(process)


In [90]:
res_df = pd.DataFrame(output_res)
res_df.to_csv('./Eva_data.csv')